In [1]:
import pandas as pd

In [2]:
# 입력받아 분석할 파일이 저장된 경로명
pre_path = r'D:\jeju_bus_data_no_leakage\tb_bus_run_state'

In [3]:
# 입력받아 분석할 파일의 이름
tb_name = 'tb_bus_run_state'
# 출력할 분석결과 파일의 이름
result_file_name = 'run_sped_anal'
# 출력할 분석 에러로그 파일의 이름
error_file_name = 'error_tb_bus_run_state'

In [4]:
date = '20190601'

In [5]:
# 컬럼(Column) 명 매핑
# Source 상     | Excel File 상
# --------------+-------------------
event_date		= 'event_date'
event_cd		= 'event_code'
run_dist		= 'run_distance'
route_id		= 'route_id'
route_run_cnt 	= 'route_run_count'
vh_id 			= 'vehicle_id'
bus_run_cnt 	= 'bus_run_count'
station_ord 	= 'station_order'

# 단순 상수
run_sped		= 'run_sped'

In [6]:
# 분석할 파일 -> 불러와 -> 데이터프레임으로 저장.
import_file_name = pre_path + '/' + tb_name + '_' + date + '.xls'
df = pd.read_excel(import_file_name, usecols = [event_date, event_cd, run_dist, route_id, 
                                                route_run_cnt, vh_id, bus_run_cnt, station_ord])

In [7]:
# 분석결과 기록 파일 생성
r_name = pre_path + '/' + result_file_name + '_' + date + '.csv'
r_cols = [event_date, event_date + '2', station_ord, station_ord + '2',
          'dist', 'run_sped', route_id, route_run_cnt, vh_id, bus_run_cnt]
r_header = pd.DataFrame(columns = r_cols)
r_header.to_csv(r_name, index = False)

In [8]:
# 에러 기록 파일(.csv) 생성

e_name = pre_path + '/' + error_file_name + '_' + date + '.csv'
e_cols = ['date', route_id, route_run_cnt, vh_id, bus_run_cnt, 'error_code']
e_header = pd.DataFrame(columns = e_cols)
e_header.to_csv(e_name, index = False)

In [9]:
# 출발-도착 기록만 추출
df = df.query(event_cd + ' in (17, 18)')  # 18: 출발, 17: 도착

# 노선아이디, 노선운행회차로 grouping -> 버스별 운행 기록 군집 목적.
grouped = df.groupby(by = [route_id, route_run_cnt, vh_id, bus_run_cnt])

In [10]:
# 오류 메시지 출력 함수
def printError(state, error_code):
    global date, e_name
    msg = '%s\t%s\t%s' % (date, state, error_code)
    print('error', msg)
    temp_df = pd.DataFrame({'date': date, route_id: state[0], route_run_cnt: state[1],
                            vh_id: state[2], bus_run_cnt: state[3], 'error_code': error_code}, index=[0])
    # index=[0] 지우면... ValueError: If using all scalar values, you must pass an index 에러 발생...
    temp_df.to_csv(e_name, index = False, header = False, mode = 'a', encoding = 'ansi')

In [11]:
#ctr = 0

for name, group in grouped:
    #if ctr == 5:
    #    break
    
    sorted_group = group.sort_values(by = [event_date, event_cd])
    sorted_group_groupby_station_ord = sorted_group.groupby(by = station_ord)
    # BIS_ERROR_2 -> Just Notice... There's no way to tidy...
    if list(sorted_group_groupby_station_ord)[0][0] in (0, 1):
        # 첫 번째 행의 station_ord가 0 혹은 1이 아닌 경우... 
        printError(name, 'BIS_ERROR_2')
        
    # BIS_ERROR_3 -> Notice & Delete
    for s_ord, gr in sorted_group_groupby_station_ord:
        if len(gr) > 2:
            printError(name, 'BIS_ERROR_3')
            sorted_group = sorted_group.drop(gr.index[:-1])
    
    flag = True
    sorted_group_groupby_station_ord = sorted_group.groupby(by = station_ord)
    
    # BIS_ERROR_5 -> Notice & Tidying
    for s_ord, gr in sorted_group_groupby_station_ord:
        if len(gr) == 1 and flag:
            flag = False
            printError(name, 'BIS_ERROR_5')
        
        if len(gr) == 1:
            temp = gr.loc[gr.index[0]]
            temp[event_cd] = 17 if gr.loc[gr.index[0]][event_cd] == 18 else 18
            sorted_group = sorted_group.append(temp)
    
    sorted_group = sorted_group.sort_values(by = [event_date, event_cd])
    sorted_group = sorted_group.reset_index(drop=True)
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html
    
    # print('=' * 50) ; print(sorted_group) ; print('=' * 50)
    
    # 첫 행(event_cd = 17)과 마지막 행(event_cd = 18) 삭제
    sorted_group = sorted_group.drop([sorted_group.index[0], sorted_group.index[-1]])
    
    df_18 = sorted_group.query(event_cd + ' == 18').reset_index()
    df_17 = sorted_group.query(event_cd + ' == 17').reset_index()
    del df_17[route_id], df_17[route_run_cnt], df_17[vh_id], df_17[bus_run_cnt]
    df_17.columns = df_17.columns + '2'
    df_18[df_17.columns] = df_17
    
    df_18.loc[:, event_date + 'temp'] = df_18[event_date].astype('datetime64')
    df_18.loc[:, event_date + 'temp2'] = df_18[event_date + '2'].astype('datetime64')
    
    df_18['dist'] = df_18[run_dist + '2'] - df_18[run_dist]
    df_18['run_sped'] = (df_18['dist'] / \
                         ((df_18[event_date + 'temp2'].dt.hour * 3600 + \
                           df_18[event_date + 'temp2'].dt.minute * 60 + \
                           df_18[event_date + 'temp2'].dt.second) -
                          (df_18[event_date + 'temp'].dt.hour * 3600 + \
                           df_18[event_date + 'temp'].dt.minute * 60 + \
                           df_18[event_date + 'temp'].dt.second)) * 3.6).round(1)
    
    #print('=' * 50) ; print(df_18) ; print('=' * 50)
    
    df_18[r_cols].to_csv(r_name, index = False, header = False, mode = 'a')
    #ctr += 1

error 20190601	(405130001, 2, 7996027, 1)	BIS_ERROR_5
error 20190601	(405130001, 3, 7996028, 1)	BIS_ERROR_5
error 20190601	(405130001, 4, 7996025, 1)	BIS_ERROR_5


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


error 20190601	(405130001, 5, 7996007, 2)	BIS_ERROR_5
error 20190601	(405130001, 8, 7996023, 2)	BIS_ERROR_5
error 20190601	(405130001, 9, 7996026, 3)	BIS_ERROR_5
error 20190601	(405130001, 10, 7996027, 4)	BIS_ERROR_2
error 20190601	(405130001, 10, 7996027, 4)	BIS_ERROR_3
error 20190601	(405130001, 10, 7996027, 4)	BIS_ERROR_5
error 20190601	(405130001, 11, 7996028, 3)	BIS_ERROR_5
error 20190601	(405130001, 12, 7996025, 3)	BIS_ERROR_5
error 20190601	(405130001, 13, 7996007, 4)	BIS_ERROR_2
error 20190601	(405130001, 13, 7996007, 4)	BIS_ERROR_5
error 20190601	(405130001, 14, 7996017, 4)	BIS_ERROR_2
error 20190601	(405130001, 14, 7996017, 4)	BIS_ERROR_3
error 20190601	(405130001, 14, 7996017, 4)	BIS_ERROR_5
error 20190601	(405130001, 15, 7996024, 5)	BIS_ERROR_5
error 20190601	(405130001, 16, 7996023, 4)	BIS_ERROR_5
error 20190601	(405130001, 18, 7996027, 6)	BIS_ERROR_5
error 20190601	(405130001, 23, 7996024, 7)	BIS_ERROR_5
error 20190601	(405130001, 25, 7996026, 8)	BIS_ERROR_5
error 2019060

error 20190601	(405131502, 11, 7983174, 6)	BIS_ERROR_5
error 20190601	(405131502, 12, 7983173, 4)	BIS_ERROR_5
error 20190601	(405131502, 14, 7983176, 4)	BIS_ERROR_5
error 20190601	(405131502, 15, 7983154, 5)	BIS_ERROR_5
error 20190601	(405131502, 16, 7983155, 6)	BIS_ERROR_5
error 20190601	(405131502, 17, 7983157, 5)	BIS_ERROR_5
error 20190601	(405131502, 18, 7983174, 8)	BIS_ERROR_2
error 20190601	(405131502, 18, 7983174, 8)	BIS_ERROR_3
error 20190601	(405131502, 18, 7983174, 8)	BIS_ERROR_5
error 20190601	(405131502, 19, 7983175, 6)	BIS_ERROR_5
error 20190601	(405131502, 20, 7983176, 6)	BIS_ERROR_5
error 20190601	(405131502, 21, 7983154, 7)	BIS_ERROR_5
error 20190601	(405131502, 22, 7983155, 8)	BIS_ERROR_5
error 20190601	(405131503, 2, 7983174, 2)	BIS_ERROR_5
error 20190601	(405131504, 1, 7983154, 1)	BIS_ERROR_5
error 20190601	(405131507, 1, 7983176, 7)	BIS_ERROR_5
error 20190601	(405131508, 1, 7983157, 7)	BIS_ERROR_2
error 20190601	(405131508, 1, 7983157, 7)	BIS_ERROR_3
error 20190601	

error 20190601	(405132512, 16, 7983532, 7)	BIS_ERROR_5
error 20190601	(405132512, 17, 7983564, 5)	BIS_ERROR_5
error 20190601	(405132512, 18, 7983524, 5)	BIS_ERROR_5
error 20190601	(405132512, 19, 7983516, 9)	BIS_ERROR_5
error 20190601	(405132512, 20, 7983504, 7)	BIS_ERROR_5
error 20190601	(405132512, 22, 7983562, 8)	BIS_ERROR_5
error 20190601	(405132513, 1, 7983527, 1)	BIS_ERROR_5
error 20190601	(405132514, 1, 7983527, 8)	BIS_ERROR_5
error 20190601	(405132514, 2, 7983532, 10)	BIS_ERROR_5
error 20190601	(405132515, 1, 7983562, 1)	BIS_ERROR_5
error 20190601	(405132601, 1, 7983536, 1)	BIS_ERROR_5
error 20190601	(405132601, 4, 7983563, 3)	BIS_ERROR_5
error 20190601	(405132601, 5, 7983536, 6)	BIS_ERROR_5
error 20190601	(405132601, 6, 7983525, 4)	BIS_ERROR_5
error 20190601	(405132601, 7, 7983563, 5)	BIS_ERROR_5
error 20190601	(405132601, 8, 7983562, 6)	BIS_ERROR_5
error 20190601	(405132601, 9, 7983532, 8)	BIS_ERROR_5
error 20190601	(405132602, 1, 7983525, 1)	BIS_ERROR_5
error 20190601	(40513

...

error 20190601	(406153101, 0, 7997563, 4)	BIS_ERROR_5
error 20190601	(406153101, 1, 7997563, 2)	BIS_ERROR_5
error 20190601	(406153101, 2, 7997565, 2)	BIS_ERROR_5
error 20190601	(406153101, 3, 7997559, 2)	BIS_ERROR_5
error 20190601	(406153101, 4, 7997548, 2)	BIS_ERROR_2
error 20190601	(406153101, 4, 7997548, 2)	BIS_ERROR_5
error 20190601	(406153101, 6, 7997565, 4)	BIS_ERROR_5
error 20190601	(406153101, 7, 7997539, 4)	BIS_ERROR_5
error 20190601	(406153101, 8, 7997562, 4)	BIS_ERROR_5
error 20190601	(406153101, 9, 7997564, 7)	BIS_ERROR_5
error 20190601	(406153101, 10, 7997559, 6)	BIS_ERROR_5
error 20190601	(406153101, 11, 7997539, 6)	BIS_ERROR_5
error 20190601	(406153101, 13, 7997563, 6)	BIS_ERROR_5
error 20190601	(406153101, 14, 7997562, 6)	BIS_ERROR_5
error 20190601	(406153101, 18, 7997562, 8)	BIS_ERROR_5
error 20190601	(406153102, 1, 7997565, 1)	BIS_ERROR_5
error 20190601	(406153102, 3, 7997563, 3)	BIS_ERROR_5
error 20190601	(406153102, 4, 7997564, 4)	BIS_ERROR_5
error 20190601	(4061531

error 20190601	(406262201, 6, 7996615, 10)	BIS_ERROR_5
error 20190601	(406262202, 2, 7996615, 4)	BIS_ERROR_5
error 20190601	(406262202, 4, 7996615, 9)	BIS_ERROR_5
error 20190601	(406262202, 5, 7996615, 11)	BIS_ERROR_5
error 20190601	(406262303, 1, 7997524, 1)	BIS_ERROR_5
error 20190601	(406262303, 3, 7997525, 5)	BIS_ERROR_5
error 20190601	(406262304, 1, 7997524, 2)	BIS_ERROR_5
error 20190601	(406262304, 2, 7997525, 2)	BIS_ERROR_5
error 20190601	(406262304, 3, 7997524, 3)	BIS_ERROR_5
error 20190601	(406262304, 4, 7997525, 3)	BIS_ERROR_2
error 20190601	(406262304, 4, 7997525, 3)	BIS_ERROR_3
error 20190601	(406262304, 4, 7997525, 3)	BIS_ERROR_5
error 20190601	(406262304, 5, 7997524, 4)	BIS_ERROR_5
error 20190601	(406262304, 8, 7997525, 6)	BIS_ERROR_5
error 20190601	(406262305, 1, 7997525, 1)	BIS_ERROR_5
error 20190601	(406262403, 1, 7997541, 2)	BIS_ERROR_2
error 20190601	(406262403, 1, 7997541, 2)	BIS_ERROR_5
error 20190601	(406262404, 1, 7997540, 2)	BIS_ERROR_5
error 20190601	(406262404,

error 20190601	(406264301, 3, 7983214, 2)	BIS_ERROR_5
error 20190601	(406264301, 4, 7983212, 4)	BIS_ERROR_5
error 20190601	(406264301, 5, 7983215, 6)	BIS_ERROR_5
error 20190601	(406264301, 6, 7983214, 6)	BIS_ERROR_5
error 20190601	(406264301, 7, 7983216, 4)	BIS_ERROR_5
error 20190601	(406264301, 8, 7983212, 8)	BIS_ERROR_5
error 20190601	(406264302, 0, 7983216, 2)	BIS_ERROR_5
error 20190601	(406264302, 2, 7983214, 4)	BIS_ERROR_5
error 20190601	(406264302, 3, 7983212, 6)	BIS_ERROR_5
error 20190601	(406264402, 1, 7983212, 1)	BIS_ERROR_5
error 20190601	(406264402, 2, 7983215, 1)	BIS_ERROR_5
error 20190601	(406264402, 4, 7983212, 3)	BIS_ERROR_5
error 20190601	(406264402, 6, 7983214, 3)	BIS_ERROR_5
error 20190601	(406264402, 7, 7983216, 2)	BIS_ERROR_5
error 20190601	(406264402, 8, 7983212, 5)	BIS_ERROR_5
error 20190601	(406264402, 9, 7983215, 5)	BIS_ERROR_5
error 20190601	(406264402, 10, 7983214, 5)	BIS_ERROR_5
error 20190601	(406264402, 11, 7983216, 3)	BIS_ERROR_5
error 20190601	(406264402,

error 20190601	(406269002, 2, 7996626, 3)	BIS_ERROR_5
error 20190601	(406269002, 5, 7996633, 6)	BIS_ERROR_5
error 20190601	(406269002, 6, 7996626, 7)	BIS_ERROR_5
error 20190601	(406269002, 7, 7996633, 8)	BIS_ERROR_5
error 20190601	(406269002, 8, 7996626, 8)	BIS_ERROR_5
error 20190601	(406269002, 9, 7996633, 10)	BIS_ERROR_5
error 20190601	(406269003, 1, 7996626, 1)	BIS_ERROR_2
error 20190601	(406269003, 1, 7996626, 1)	BIS_ERROR_5
error 20190601	(406269101, 1, 7996639, 1)	BIS_ERROR_5
error 20190601	(406269102, 1, 7996637, 1)	BIS_ERROR_5
error 20190601	(406269103, 1, 7996639, 2)	BIS_ERROR_5
error 20190601	(406269201, 1, 7996638, 1)	BIS_ERROR_5
error 20190601	(406329501, 1, 7983037, 1)	BIS_ERROR_2
error 20190601	(406329501, 1, 7983037, 1)	BIS_ERROR_5
error 20190601	(406329501, 2, 7983055, 1)	BIS_ERROR_2
error 20190601	(406329501, 2, 7983055, 1)	BIS_ERROR_3
error 20190601	(406329501, 2, 7983055, 1)	BIS_ERROR_5
error 20190601	(406329501, 3, 7983008, 2)	BIS_ERROR_5
error 20190601	(406329501, 

error 20190601	(406574202, 5, 7996605, 5)	BIS_ERROR_5
error 20190601	(406574202, 6, 7996605, 6)	BIS_ERROR_5
error 20190601	(406574202, 8, 7996605, 8)	BIS_ERROR_5
error 20190601	(406574203, 1, 7996607, 1)	BIS_ERROR_5
error 20190601	(406575101, 1, 7996624, 2)	BIS_ERROR_5
error 20190601	(406575101, 2, 7996624, 4)	BIS_ERROR_5
error 20190601	(406575101, 3, 7996624, 6)	BIS_ERROR_5
error 20190601	(406575102, 1, 7996624, 1)	BIS_ERROR_5
error 20190601	(406575102, 2, 7996624, 3)	BIS_ERROR_5
error 20190601	(406575102, 5, 7996624, 9)	BIS_ERROR_5
error 20190601	(406575102, 6, 7996624, 11)	BIS_ERROR_5
error 20190601	(406575103, 1, 7996630, 2)	BIS_ERROR_2
error 20190601	(406575103, 1, 7996630, 2)	BIS_ERROR_5
error 20190601	(406575103, 3, 7996630, 6)	BIS_ERROR_5
error 20190601	(406575104, 0, 7996631, 0)	BIS_ERROR_5
error 20190601	(406575104, 1, 7996631, 2)	BIS_ERROR_5
error 20190601	(406575104, 2, 7996631, 4)	BIS_ERROR_5
error 20190601	(406575104, 3, 7996631, 6)	BIS_ERROR_5
error 20190601	(406575107, 